In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'bucket-esp1'
directory = 'BigData1'

## READ METADATA
---------
Download files for cleaning tables.

In [2]:
raw_data1 = get_input_data(
    bucket= bucket,
    initial_directory= directory,
    filename = 'datos-abiertos-agosto-2019.csv')
print(raw_data1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18141 entries, 0 to 18140
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18141 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  18141 non-null  object
 2   CODIGO DE LOCALIDAD                18141 non-null  int64 
 3   LOCALIDAD                          18141 non-null  object
 4   EDAD                               18141 non-null  object
 5   UNIDAD                             18141 non-null  object
 6   GENERO                             18141 non-null  object
 7   RED                                18141 non-null  object
 8   TIPO DE INCIDIDENT                 18141 non-null  object
 9   PRIORIDAD                          18141 non-null  object
 10  MES                                18141 non-null  object
 11  CLASIFICACION FINAL                18138 non-null  object
dtypes: i

In [3]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data1.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data1.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO DE INCIDIDENT,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [4]:
raw_data1['LOCALIDAD'].value_counts(dropna=False)

Kennedy                2717
 Engativa              1805
 Suba                  1678
 Bosa                  1264
Ciudad Bolivar         1220
Puente Aranda           977
 San Cristobal          906
Usaquen                 894
 Rafael Uribe Uribe     892
Fontibon                878
Chapinero               707
Santa Fe                682
 Usme                   637
 Barrios Unidos         608
 Teusaquillo            599
Tunjuelito              595
Los Martires            593
 Antonio Nariño         369
 La Candelaria          119
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [5]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data1['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2717
Engativa              1805
Suba                  1678
Bosa                  1264
Ciudad Bolivar        1220
Puente Aranda          977
San Cristobal          906
Usaquen                894
Rafael Uribe Uribe     892
Fontibon               878
Chapinero              707
Santa Fe               682
Usme                   637
Barrios Unidos         608
Teusaquillo            599
Tunjuelito             595
Los Martires           593
Antonio Nariño         369
La Candelaria          119
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [6]:
raw_data1['LOCALIDAD'] = raw_data1['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [7]:
raw_data1.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO DE INCIDIDENT,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [8]:
col = 'GENERO'
raw_data1[col] = raw_data1[col].apply(lambda x: x.strip())
raw_data1[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     8617
MASCULINO    5143
FEMENINO     4381
Name: GENERO, dtype: int64

In [9]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data1[col].value_counts(dropna=False)

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
NaN                               3
Name: CLASIFICACION_FINAL, dtype: int64

In [10]:
raw_data1[col].fillna('SIN_DATO').value_counts()

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
SIN_DATO                          3
Name: CLASIFICACION_FINAL, dtype: int64

In [11]:
col = 'CLASIFICACION_FINAL'
raw_data1[col].fillna('SIN_DATO', inplace=True)
raw_data1[col] = raw_data1[col].apply(lambda x: x.strip())
raw_data1.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO DE INCIDIDENT,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [48]:
profile = ProfileReport(df=raw_data1)
profile.to_file(output_file=project_dir + '/reports/'+'datos-abiertos-agosto-2019.csv')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
print('Number of rows before cleaning:', raw_data1.shape[0] )
raw_data1.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data1.shape[0] )

Number of rows before cleaning: 18141
Number of rows after cleaning: 16807


In [53]:
# saving in a local directory
raw_data1.to_csv(project_dir + '/data/processed/' +'datos-abiertos-agosto-2019.csv', encoding='latin1', sep=';', index=False)

In [97]:
raw_data3 = get_input_data(
    bucket= bucket,
    initial_directory= directory,
    filename = 'datos-abiertos-septiembre-2019.csv')
print(raw_data3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17018 entries, 0 to 17017
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   17018 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  17018 non-null  object
 2   CODIGO DE LOCALIDAD                17018 non-null  int64 
 3   LOCALIDAD                          17018 non-null  object
 4   EDAD                               17018 non-null  int64 
 5   UNIDAD                             9371 non-null   object
 6   GENERO                             17018 non-null  object
 7   RED                                17018 non-null  object
 8   TIPO_INCIDENTE                     17018 non-null  object
 9   PRIORIDAD                          17018 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.3+ MB
None


In [98]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data3.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data3.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [99]:
raw_data3['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2398
 Suba                  1634
Engativa               1631
 Bosa                  1294
 Ciudad Bolivar        1138
 Puente Aranda          938
 Rafael Uribe Uribe     854
 Usaquen                829
 San Cristobal          826
Fontibon                809
 Chapinero              749
 Barrios Unidos         616
Tunjuelito              603
Los Martires            593
Usme                    572
 Santa Fe               549
 Teusaquillo            540
 Antonio Nariño         362
 La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [100]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data3['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2398
Suba                  1634
Engativa              1631
Bosa                  1294
Ciudad Bolivar        1138
Puente Aranda          938
Rafael Uribe Uribe     854
Usaquen                829
San Cristobal          826
Fontibon               809
Chapinero              749
Barrios Unidos         616
Tunjuelito             603
Los Martires           593
Usme                   572
Santa Fe               549
Teusaquillo            540
Antonio Nariño         362
La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [101]:
raw_data3['LOCALIDAD'] = raw_data3['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column
raw_data3.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [40]:
col = 'GENERO'
raw_data3[col] = raw_data3[col].apply(lambda x: x.strip())
raw_data3[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     7632
MASCULINO    5082
FEMENINO     4304
Name: GENERO, dtype: int64

In [67]:
col = 'TIPO_INCIDENTE'
raw_data3[col] = raw_data3[col].apply(lambda x: x.strip())
raw_data3[col].value_counts(dropna=False)

ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS    6333
INCONSCIENTE/PARO CARDIORRESPIRATORIO        1720
DIFICULTAD RESPIRATORIA                      1386
ENFERMO                                      1137
CONVULSIONES                                 1039
HERIDOS                                       834
INTENTO DE SUICIDIO                           653
DOLOR TORÁCICO                                629
TRASTORNO MENTAL                              621
LESIONES PERSONALES                           593
PATOLOGÍA GINECO - OBSTÉTRICA                 421
SÍNTOMAS GASTROINTESTINALES                   360
ACCIDENTE CEREBRO VASCULAR                    352
CAÍDA                                         334
INTOXICACIONES                                156
IDEAS DE SUICIDIO                             113
VIOLENCIA SEXUAL                               48
INCENDIO ESTRUCTURAL                           40
SIN_DATO                                       32
SANGRADO VAGINAL                               25


In [70]:
profile = ProfileReport(df=raw_data3)
profile.to_file(output_file=project_dir + '/reports/'+' datos-abiertos-septiembre-2019.csv')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
print('Number of rows before cleaning:', raw_data3.shape[0] )
raw_data3.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data3.shape[0] )

Number of rows before cleaning: 17018
Number of rows after cleaning: 16267


In [72]:
# saving in a local directory
raw_data3.to_csv(project_dir + '/data/processed/' +'datos-abiertos-septiembre-2019.csv', encoding='latin1', sep=';', index=False)

In [74]:
raw_data4 = get_input_data(
    bucket= bucket,
    initial_directory= directory,
    filename = 'datos-abiertos-julio-2019.csv')
print(raw_data4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14560 entries, 0 to 14559
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INCIDENTE                    14560 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  14560 non-null  object
 2   CODIGO DE LOCALIDAD                14560 non-null  int64 
 3   LOCALIDAD                          14560 non-null  object
 4   EDAD                               14560 non-null  object
 5   UNIDAD                             14560 non-null  object
 6   GENERO                             14560 non-null  object
 7   RED                                14560 non-null  object
 8   TIPO_INCIDENTE                     14560 non-null  object
 9   PRIORIDAD                          14560 non-null  object
 10  MES                                14560 non-null  object
 11  CLASIFICACION FINAL                13035 non-null  object
dtypes: i

In [76]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data4.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data4.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [77]:
raw_data4['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               1593
 Suba                  1548
Engativa               1521
 Bosa                  1122
 Ciudad Bolivar         994
Usaquen                 786
San Cristobal           776
 Rafael Uribe Uribe     741
Fontibon                723
 Puente Aranda          690
Usme                    636
Chapinero               599
Barrios Unidos          516
Los Martires            516
Santa Fe                489
Teusaquillo             464
Tunjuelito              462
 Antonio Nariño         292
 La Candelaria           91
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [78]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data4['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               1593
Suba                  1548
Engativa              1521
Bosa                  1122
Ciudad Bolivar         994
Usaquen                786
San Cristobal          776
Rafael Uribe Uribe     741
Fontibon               723
Puente Aranda          690
Usme                   636
Chapinero              599
Barrios Unidos         516
Los Martires           516
Santa Fe               489
Teusaquillo            464
Tunjuelito             462
Antonio Nariño         292
La Candelaria           91
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [79]:
raw_data4['LOCALIDAD'] = raw_data4['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column
raw_data4.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [80]:
col = 'GENERO'
raw_data4[col] = raw_data4[col].apply(lambda x: x.strip())
raw_data4[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     6621
MASCULINO    4190
FEMENINO     3749
Name: GENERO, dtype: int64

In [81]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data4[col].value_counts(dropna=False)

Traslado                       6162
Cancelado                      3637
NaN                            1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

In [ ]:
raw_data[col].fillna('SIN_DATO').value_counts()

In [82]:
col = 'CLASIFICACION_FINAL'
raw_data4[col].fillna('SIN_DATO', inplace=True)
raw_data4[col] = raw_data4[col].apply(lambda x: x.strip())
raw_data4.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [83]:
profile = ProfileReport(df=raw_data4)
profile.to_file(output_file=project_dir + '/reports/'+'datos-abiertos-julio-2019.csv')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [84]:
print('Number of rows before cleaning:', raw_data4.shape[0] )
raw_data4.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data4.shape[0] )

Number of rows before cleaning: 14560
Number of rows after cleaning: 13842


In [85]:
# saving in a local directory
raw_data4.to_csv(project_dir + '/data/processed/' +'datos-abiertos-julio-2019.csv', encoding='latin1', sep=';', index=False)

In [28]:
raw_data5 = get_input_data(
    bucket= bucket,
    initial_directory= directory,
    filename = 'datos-abiertos-noviembre_2019.csv')
print(raw_data5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15926 entries, 0 to 15925
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   15926 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  15926 non-null  object
 2   CODIGO DE LOCALIDAD                15926 non-null  int64 
 3   LOCALIDAD                          15926 non-null  object
 4   EDAD                               15926 non-null  int64 
 5   UNIDAD                             15926 non-null  object
 6   GENERO                             15926 non-null  object
 7   RED                                15926 non-null  object
 8   TIPO_INCIDENTE                     15926 non-null  object
 9   PRIORIDAD                          15926 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


In [86]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data5.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data5.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [87]:
raw_data5['LOCALIDAD'].value_counts(dropna=False)

 Kennedy              2192
Engativa              1571
Suba                  1424
 Bosa                 1189
 Ciudad Bolivar       1123
 Puente Aranda         854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
 Santa Fe              613
 Chapinero             577
 Usme                  577
Los Martires           560
 Tunjuelito            551
 Teusaquillo           546
 Barrios Unidos        520
 Antonio Nariño        338
 La Candelaria         102
 Sumapaz                 1
Name: LOCALIDAD, dtype: int64

In [88]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data5['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2192
Engativa              1571
Suba                  1424
Bosa                  1189
Ciudad Bolivar        1123
Puente Aranda          854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
Santa Fe               613
Chapinero              577
Usme                   577
Los Martires           560
Tunjuelito             551
Teusaquillo            546
Barrios Unidos         520
Antonio Nariño         338
La Candelaria          102
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [89]:
raw_data5['LOCALIDAD'] = raw_data5['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column
raw_data5.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [90]:
col = 'GENERO'
raw_data5[col] = raw_data5[col].apply(lambda x: x.strip())
raw_data5[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     7011
MASCULINO    4856
FEMENINO     4059
Name: GENERO, dtype: int64

In [93]:
profile = ProfileReport(df=raw_data5)
profile.to_file(output_file=project_dir + '/reports/'+'datos-abiertos-noviembre_2019.csv')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [94]:
print('Number of rows before cleaning:', raw_data5.shape[0] )
raw_data5.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data5.shape[0] )

Number of rows before cleaning: 15926
Number of rows after cleaning: 15886


In [96]:
# saving in a local directory
raw_data5.to_csv(project_dir + '/data/processed/' +'datos-abiertos-noviembre_2019.csv', encoding='latin1', sep=';', index=False)

In [30]:
raw_data6 = get_input_data(
    bucket= bucket,
    initial_directory= directory,
    filename = 'datos_abiertos_cons_diciembre_2019.csv')
print(raw_data1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18141 entries, 0 to 18140
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18141 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  18141 non-null  object
 2   CODIGO_LOCALIDAD                   18141 non-null  int64 
 3   LOCALIDAD                          18141 non-null  object
 4   EDAD                               18141 non-null  object
 5   UNIDAD                             18141 non-null  object
 6   GENERO                             18141 non-null  object
 7   RED                                18141 non-null  object
 8   TIPO DE INCIDIDENT                 18141 non-null  object
 9   PRIORIDAD                          18141 non-null  object
 10  MES                                18141 non-null  object
 11  CLASIFICACION_FINAL                18141 non-null  object
dtypes: i

In [102]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data6.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data6.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1178693196,2019-12-01 03:18:23,8,Kennedy,14,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
1,1176480196,2019-12-01 00:05:59,5,Usme,0,SIN/DATO,SIN/DATO,Sur,Accidente Tránsito con Heridos/Muertos,ALTA
2,1180282196,2019-12-01 05:37:10,7,Bosa,32,Años,MASCULINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA
3,1176520196,2019-12-01 00:07:43,3,Santa Fe,0,SIN/DATO,SIN/DATO,Norte,Lesiones Personales,ALTA
4,1180873196,2019-12-01 07:14:08,5,Usme,53,Años,FEMENINO,Privada,Accidente Tránsito con Heridos/Muertos,ALTA


In [103]:
raw_data6['LOCALIDAD'].value_counts(dropna=False)

 Kennedy              2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [104]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data6['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2317
Engativá              1644
Suba                  1471
Bosa                  1182
Ciudad Bolívar        1121
Puente Aranda          904
Fontibón               864
San Cristóbal          838
Rafael Uribe Uribe     772
Usaquén                770
Usme                   630
Tunjuelito             616
Santa Fe               589
Chapinero              556
Los Mártires           525
Barrios Unidos         513
Teusaquillo            497
Antonio Nariño         373
La Candelaria           60
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [109]:
col = 'GENERO'
raw_data6[col] = raw_data6[col].apply(lambda x: x.strip())
raw_data6[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN/DATO     8286
MASCULINO    4405
FEMENINO     3552
Name: GENERO, dtype: int64

In [110]:
profile = ProfileReport(df=raw_data6)
profile.to_file(output_file=project_dir + '/reports/'+'datos_abiertos_cons_diciembre_2019.csv')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [111]:
print('Number of rows before cleaning:', raw_data6.shape[0] )
raw_data6.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data6.shape[0] )

Number of rows before cleaning: 16243
Number of rows after cleaning: 16162


In [112]:
# saving in a local directory
raw_data6.to_csv(project_dir + '/data/processed/' +'datos_abiertos_cons_diciembre_2019.csv', encoding='latin1', sep=';', index=False)

In [114]:
raw_data7 = get_input_data(
    bucket= bucket,
    initial_directory= directory,
    filename = 'datos-abiertos-octubre-2019.csv')
print(raw_data7.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     18195 non-null  int64 
 1   FECHA_DESPACHO_518   18195 non-null  object
 2   CODIGO DE LOCALIDAD  18195 non-null  int64 
 3   LOCALIDAD            18195 non-null  object
 4   EDAD                 18195 non-null  object
 5   UNIDAD               9503 non-null   object
 6   GENERO               9543 non-null   object
 7   RED                  18195 non-null  object
 8   TIPO_INCIDENTE       18195 non-null  object
 9   PRIORIDAD            18195 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.4+ MB
None


In [115]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data7.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data7.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [116]:
raw_data7['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2546
 Engativa              1791
 Suba                  1742
 Bosa                  1334
 Ciudad Bolivar        1141
 Puente Aranda          989
 Rafael Uribe Uribe     949
Usaquen                 888
 Fontibon               863
San Cristobal           844
 Chapinero              763
 Los Martires           697
 Santa Fe               689
 Barrios Unidos         680
 Usme                   609
 Tunjuelito             589
 Teusaquillo            536
 Antonio Nariño         419
 La Candelaria          120
 Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [117]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data7['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2546
Engativa              1791
Suba                  1742
Bosa                  1334
Ciudad Bolivar        1141
Puente Aranda          989
Rafael Uribe Uribe     949
Usaquen                888
Fontibon               863
San Cristobal          844
Chapinero              763
Los Martires           697
Santa Fe               689
Barrios Unidos         680
Usme                   609
Tunjuelito             589
Teusaquillo            536
Antonio Nariño         419
La Candelaria          120
Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [119]:
raw_data7['LOCALIDAD'] = raw_data7['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column
raw_data7.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [121]:
col = 'TIPO_INCIDENTE'
raw_data7[col] = raw_data7[col].apply(lambda x: x.strip())
raw_data7[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


In [122]:
profile = ProfileReport(df=raw_data7)
profile.to_file(output_file=project_dir + '/reports/'+'datos-abiertos-octubre-2019.csv')

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [123]:
print('Number of rows before cleaning:', raw_data7.shape[0] )
raw_data7.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data7.shape[0] )

Number of rows before cleaning: 18195
Number of rows after cleaning: 16559


In [124]:
# saving in a local directory
raw_data7.to_csv(project_dir + '/data/processed/' +'datos-abiertos-octubre-2019.csv', encoding='latin1', sep=';', index=False)

In [127]:
dfx= pd.concat([raw_data1, raw_data3, raw_data4, raw_data5, raw_data6, raw_data7])
dfx

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO DE INCIDIDENT,PRIORIDAD,MES,CLASIFICACION_FINAL,TIPO_INCIDENTE,FECHA_INCIDENTE,FECHA_DESPACHO_518
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado,NaN,NaN,NaN
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado,NaN,NaN,NaN
5,809244194,1/08/2019 0:10,13,Teusaquillo,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN,NaN,NaN
7,809253194,1/08/2019 0:12,8,Kennedy,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente de tránsito con heridos/Muertos,ALTA,AGOSTO,Cancelado,NaN,NaN,NaN
9,809261194,1/08/2019 0:13,11,Suba,71,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,AGOSTO,Traslado,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18190,1128631194,NaN,17,La Candelaria,23,Años,MASCULINO,Norte,NaN,SIN_DATO,NaN,NaN,SIN_DATO,NaN,2019-10-31 23:23:45
18191,1128688194,NaN,19,Ciudad Bolivar,13,Años,MASCULINO,Sur,NaN,SIN_DATO,NaN,NaN,SIN_DATO,NaN,2019-10-31 23:29:29
18192,1162693195,NaN,1,Usaquen,SIN_DATO,NaN,NaN,Norte,NaN,ALTA,NaN,NaN,Accidente de tránsito con heridos/Muertos,NaN,2019-10-18 13:33:43
18193,1085341195,NaN,9,Fontibon,17,Años,FEMENINO,Norte,NaN,ALTA,NaN,NaN,Heridos,NaN,2019-10-10 18:50:51


In [128]:
profile = ProfileReport(df=dfx)
profile.to_file(output_file=project_dir + '/reports/'+'dfx.csv')

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pandas_profiling/profile_report.py:267: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  f"Extension {suffix} not supported. For now we assume .html was intended. "


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [129]:
# saving in a local directory
dfx.to_csv(project_dir + '/data/processed/' +'clean_dfx.csv', encoding='latin1', sep=';', index=False)